In [3]:
from collections import defaultdict

In [95]:
class Node:
    """
    A node in a graph.
    Each node has pointers to its neighbours.
    """
    def __init__(self, data=None):
        self.neighbours = set()
        self.data = data
        
class Triangle:
    """
    Unordered triangle is represented as a list of its
    vertices (labeled with natural numbers).
    """
    def __init__(self, vertices):
        assert len(vertices) == 3, 'A triangle should have 3 vertices'
        self.vertices = sorted(vertices)
        
    @property
    def edges(self):
        return (
                (self.vertices[0], self.vertices[1]),
                (self.vertices[1], self.vertices[2]),
                (self.vertices[2], self.vertices[0]),
               )

    def __repr__(self):
        return str(self.edges)
        
class OrderedTriangle:
    """
    Ordered triangle points to its unordered triangle,
    its lead vertex and to its ordered neighbour that
    shares the same leading edge.
    """
    def __init__(self, triangle, i, ordered_neighbour):
        self.i = i
        self.ordered_neighbour = ordered_neighbour

In [94]:
torus = [
    (0, 1, 6),
    (0, 4, 6),
    (1, 6, 7),
    (1, 2, 7),
    (2, 7, 4),
    (2, 0, 4),
    (4, 5, 8),
    (4, 6, 8),
    (6, 3, 8),
    (6, 7, 3),
    (3, 7, 5),
    (7, 4, 5),
    (0, 1, 5),
    (1, 5, 8),
    (1, 2, 8),
    (2, 8, 3),
    (2, 0, 3),
    (0, 5, 3),
]

In [96]:
def create_triangulation(triangles):
    triangles = [Triangle(triangle) for triangle in triangles]
    n = max([max(triangle.vertices) for triangle in triangles]) + 1
    nodes = [Node(triangle) for triangle in triangles]
    # For each vertex construct a set of nodes containing 
    # triangles with this vertex.    
    vertex_nodes = defaultdict(set)
    for node in nodes:
        for vertex in node.data.vertices:
            vertex_nodes[vertex].add(node)
    # For each triangle construct a node in a graph and connect it with
    # its neighbours in the triangulation.
    for node in nodes:
        for (v1, v2) in node.data.edges:
            neighbour = (vertex_nodes[v1] & vertex_nodes[v2]) - set([node])
            assert len(neighbour) == 1
            node.neighbours.add(neighbour.pop())
        assert len(node.neighbours) == 3
    return nodes    

In [97]:
triangulation = create_triangulation(torus)

In [120]:
def get_orientation(edge, edges):
    """
    Returt the orientation of edge in edges.
    It can be 0, 1 or 2 (edge is one of the edges)
    or 4, 5 or 6 (edge is in the oposite orientation as one of the edges).
    """
    try:
        index =  edges.index(edge)
    except Exception:
        index = edges.index((edge[1], edge[0])) + 4
    return index

def sym(orientation):
    return (orientation + 4) % 8

def enext(orientation):
    if orientation <= 2:
        return (orientation + 1) % 3
    return ((orientation + 1) % 3) + 4

    
def order_triangulation(triangulation):
    for node in triangulation:
        triangle = node.data
        neighbours = node.neighbours
        # Two assumptions made here:
        # - triangle vertices are ordered
        # - function edges of triangle returns edges  
        #   in the positive orientation
        ordered = dict()
        for i, edge in enumerate(triangle.edges):
            neighbour = [n for n in neighbours if edge[0] in n.data.vertices and edge[1] in n.data.vertices][0]
            orientation = get_orientation(edge, neighbour.data.edges)
            ordered[i] = (neighbour.data, orientation)
            ordered[sym(i)] = (neighbour.data, sym(orientation))
        triangle.ordered = ordered

In [106]:
order_triangulation(triangulation)

In [131]:
def orientable(ordered_triangulation):
    return is_orientable(ordered_triangulation[0].data.ordered[0])

def same_orientation_class(o1, o2):
    if o1 in [0, 1, 2] and o2 in [0, 1, 2]:
        return True
    elif o1 in [4, 5, 6] and o2 in [4, 5, 6]:
        return True
    return False

def is_orientable(oriented_triangle):
    (triangle, orientation) = oriented_triangle
    chosen = getattr(triangle, 'orientation', None)
    if chosen is None:
        triangle.orientation = 0
        (t1, o1) = triangle.ordered[sym(orientation)]
        (t2, o2) = triangle.ordered[enext(sym(orientation))]
        (t3, o3) = triangle.ordered[enext(enext(sym(orientation)))]
        return is_orientable((t1, o1)) and is_orientable((t2, o2)) and is_orientable((t3, o3))
    else:
        return same_orientation_class(orientation, chosen)

In [132]:
orientable(triangulation)

False